In [ ]:
import torch
from torch.utils.data import DataLoader

from _models import (
    CoordinateCNN,
    CoordinateDataset,
    DetectionLoss,
    plot_training_history,
    train_and_validate,
)

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize model
model = CoordinateCNN().to(device)

# Define loss function and optimizer
criterion = DetectionLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# Create dataset and dataloader
dataset: CoordinateDataset = CoordinateDataset("./train")
valDataset: CoordinateDataset = CoordinateDataset("./val")

train_loader: DataLoader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader: DataLoader = DataLoader(valDataset, batch_size=32, shuffle=True)

print(f"Number of training samples: {len(train_loader.dataset)}")
print(f"Number of validation samples: {len(val_loader.dataset)}")

In [ ]:
# Train and validate the model
history = train_and_validate(
    device,
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs=200,
    patience=20,
)

# Plot training history
plot_training_history(history)

print("Training completed!")
print(f"Best validation loss: {min(history['val_loss']):.6f}")

In [ ]:
model.load_state_dict(torch.load("models/checkpoint.pt", weights_only=True))

In [ ]:
# Evaluate on validation set
testDataset: CoordinateDataset = CoordinateDataset("./testImages")
test_loader: DataLoader = DataLoader(testDataset, batch_size=32, shuffle=True)
print(f"Number of test samples: {len(test_loader.dataset)}")

In [ ]:
from _models import evaluate_model

# Evaluate on test set with mAP metrics
test_metrics = evaluate_model(model, test_loader, device, criterion)

print(f"Test mAP50: {test_metrics['val_map50']:.2f}%")
print(f"Test loss: {test_metrics['val_loss']:.4f}")

In [ ]:
# Evaluate on train & validation set
test_metrics = evaluate_model(model, train_loader, device, criterion)

print(f"Train mAP50: {test_metrics['val_map50']:.2f}%")
print(f"Train loss: {test_metrics['val_loss']:.4f}")

test_metrics = evaluate_model(model, val_loader, device, criterion)

print(f"Val mAP50: {test_metrics['val_map50']:.2f}%")
print(f"Val loss: {test_metrics['val_loss']:.4f}")

In [ ]:
import csv
from glob import glob
from pathlib import Path
from _models import prediction_as_YOLO

# Define the path to the test images directory and output CSV file
output_csv_path = Path("./my_submission.csv")

# Ensure the model is in evaluation mode
model.eval()

# Open the CSV file for writing
with open(output_csv_path, mode="w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(["image_id", "prediction_string"])

    # Iterate over all test images
    for image_path in glob("./testImages/images/*.jpg"):  # Adjust extension if needed
        # Generate prediction for the image
        confidence, x_center, y_center, width, height = prediction_as_YOLO(
            str(image_path), model
        )

        # Write the prediction to the CSV file
        writer.writerow(
            [
                Path(image_path).name.strip(".jpg"),
                " ".join(map(str, [0, confidence, x_center, y_center, width, height])),
            ]
        )

print(f"Predictions saved to {output_csv_path}")